In [1]:
import pandas as pd
from pathlib import Path
from apriori_algo import AprioriAlgo
apriori = AprioriAlgo()

In [2]:
orders = pd.read_csv('../input/all_orders.csv', index_col = 0)
cluster_df = pd.read_csv('../input/cluster_df')
products = pd.read_csv('../input/products.csv')
aisles = pd.read_csv('../input/aisles.csv')
departments = pd.read_csv('../input/departments.csv')

c:\users\akshaysharma\hvacproakshay\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
orders = orders.sort_values(['order_id','product_id'])
orders

,order_id,product_id,add_to_cart_order,reordered,aisle_id,department_id,aisle,department,user_id,cluster
30264547,1,10246,3,0,83,4,fresh vegetables,produce,112108,1
30264545,1,11109,2,1,108,16,other creams cheeses,dairy eggs,112108,1
30264548,1,13176,6,0,24,4,fresh fruits,produce,112108,1
30264544,1,22035,8,1,21,16,packaged cheese,dairy eggs,112108,1
30264550,1,43633,5,1,95,15,canned meat seafood,canned goods,112108,1
...,...,...,...,...,...,...,...,...,...,...
23108244,3421083,21162,3,0,92,18,baby food formula,babies,25247,7
23108238,3421083,24852,9,1,24,4,fresh fruits,produce,25247,7
23108247,3421083,35211,5,0,92,18,baby food formula,babies,25247,7
23108242,3421083,39678,6,1,74,17,dish detergents,household,25247,7


In [4]:
# separate order data by cluster and transform into expected format of group association function

cluster_orders = {}
for x in range(0,9):
    i = orders[orders['cluster'] == x]
    i = i.set_index('order_id')['product_id'].rename('item_id')
    cluster_orders[x] = i

In [5]:
cluster_orders[0]

order_id
4          10054
4          17616
4          21351
4          22598
4          25146
           ...  
3421078    37646
3421078    38656
3421078    40396
3421078    41844
3421078    48577
Name: item_id, Length: 1637817, dtype: int64

In [6]:
# run each cluster series through association rule function and obtain association rules at item level
REBUILD_TEST_DATA = False
test_data_path = "../input/associationRule/group0_item_rules.csv"
if REBUILD_TEST_DATA or not Path(test_data_path).exists():
    group_association_rules_dic = {}
    for x in range(0,9):
        group_association_rules_dic[x] = apriori.association_rules(cluster_orders[x],.0001)
    for x in group_association_rules_dic:
        group_association_rules_dic[x] = group_association_rules_dic[x].merge(products[['product_id','product_name']]
                   .rename(columns = {'product_id':'item_A','product_name':'product_name_A'}), on = 'item_A')
        group_association_rules_dic[x] = group_association_rules_dic[x].merge(products[['product_id','product_name']]
                   .rename(columns = {'product_id':'item_B','product_name':'product_name_B'}), on = 'item_B')
        group_association_rules_dic[x] = group_association_rules_dic[x][['item_A','item_B','product_name_A',
                                                                        'product_name_B','freqAB','supportAB','freqA',
                                                                        'supportA','freqB','supportB','confidenceAtoB',
                                                                        'confidenceBtoA','lift']]
    for x in group_association_rules_dic:
        print("Writing cluster " + str(x) + "'s item rules to csv...")
        group_association_rules_dic[x].to_csv('../input/associationRule/group'+str(x)+'_item_rules.csv')


Starting order_item:                1637817
Items with support >= 0.0001:            9939
Remaining order_item:               1532104
Remaining orders with 2+ items:      155466
Remaining order_item:               1523912
Item pairs:                         3177970
Item pairs with support >= 0.0001:      83094

Starting order_item:               11069477
Items with support >= 0.0001:           10734
Remaining order_item:              10422194
Remaining orders with 2+ items:      889529
Remaining order_item:              10395976
Item pairs:                        12408004
Item pairs with support >= 0.0001:     102257

Starting order_item:                 521280
Items with support >= 0.0001:            1053
Remaining order_item:                518290
Remaining orders with 2+ items:       96448
Remaining order_item:                501680
Item pairs:                          147596
Item pairs with support >= 0.0001:      25595

Starting order_item:                8952464
Items with suppor

In [7]:
group_association_rules_dic[0]

,item_A,item_B,product_name_A,product_name_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
0,16576,27380,Chili Rice Scooter,Shiitake Mushroom Rice Scooter,19,0.000122,21,0.000135,22,0.000142,0.904762,0.863636,6393.623377
1,7801,38520,Multigrain Oat Manna Bread,Millet Rice Bread,19,0.000122,27,0.000174,19,0.000122,0.703704,1.000000,5758.000000
2,11109,38520,Organic 4% Milk Fat Whole Milk Cottage Cheese,Millet Rice Bread,19,0.000122,157,0.001010,19,0.000122,0.121019,1.000000,990.229299
3,22224,38520,Whole Milk Plain Greek Yogurt,Millet Rice Bread,16,0.000103,274,0.001762,19,0.000122,0.058394,0.842105,477.805609
4,7801,44517,Multigrain Oat Manna Bread,Organic Calimyrna Figs,26,0.000167,27,0.000174,66,0.000425,0.962963,0.393939,2268.303030
...,...,...,...,...,...,...,...,...,...,...,...,...,...
83089,24852,44912,Banana,Baking Powder,18,0.000116,29534,0.189971,113,0.000727,0.000609,0.159292,0.838508
83090,24852,34503,Banana,Grape-Nuts Cereal,24,0.000154,29534,0.189971,155,0.000997,0.000813,0.154839,0.815066
83091,24852,35855,Banana,Unbleached All-Purpose Flour,25,0.000161,29534,0.189971,187,0.001203,0.000846,0.133690,0.703739
83092,24852,36675,Banana,"Kale Krunch, Organic, Southwest Ranch",16,0.000103,29534,0.189971,122,0.000785,0.000542,0.131148,0.690356


In [8]:
i1 = orders.set_index('order_id')['product_id'].rename('item_id')

In [9]:
i1.head()

order_id
1    10246
1    11109
1    13176
1    22035
1    43633
Name: item_id, dtype: int64

In [10]:
association_rule_no_clustering = apriori.association_rules(i1,.0001)

Starting order_item:               33819106
Items with support >= 0.0001:           10949
Remaining order_item:              31116451
Remaining orders with 2+ items:     3136021
Remaining order_item:              30928016
Item pairs:                        21061284
Item pairs with support >= 0.0001:      61832



In [11]:
association_rule_no_clustering

,item_A,item_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
59856,41349,49570,317,0.000101,656,0.000209,602,0.000192,0.483232,0.526578,2517.316914
46905,19244,26488,326,0.000104,658,0.000210,731,0.000233,0.495441,0.445964,2125.461740
54852,11224,39739,416,0.000133,687,0.000219,987,0.000315,0.605531,0.421479,1923.970475
58537,7657,19513,338,0.000108,714,0.000228,924,0.000295,0.473389,0.365801,1606.665542
53784,29126,36361,539,0.000172,1215,0.000387,878,0.000280,0.443621,0.613895,1584.517111
...,...,...,...,...,...,...,...,...,...,...,...
28189,5450,47209,423,0.000135,51986,0.016577,220055,0.070170,0.008137,0.001922,0.115958
12466,6184,24852,549,0.000175,31103,0.009918,488729,0.155844,0.017651,0.001123,0.113261
45806,16797,21137,1415,0.000451,148254,0.047275,274267,0.087457,0.009544,0.005159,0.109133
9752,47209,47766,941,0.000300,220055,0.070170,183626,0.058554,0.004276,0.005125,0.073030


In [12]:
association_rule_no_clustering = association_rule_no_clustering.merge(aisles[['aisle_id','aisle']]
                   .rename(columns = {'aisle_id':'item_A','aisle':'aisle_name_A'}), on = 'item_A')
association_rule_no_clustering = association_rule_no_clustering.merge(aisles[['aisle_id','aisle']]
                   .rename(columns = {'aisle_id':'item_B','aisle':'aisle_name_B'}), on = 'item_B')
association_rule_no_clustering = association_rule_no_clustering[['item_A','item_B','aisle_name_A',
                                                                        'aisle_name_B','freqAB','supportAB','freqA',
                                                                        'supportA','freqB','supportB','confidenceAtoB',
                                                                        'confidenceBtoA','lift']]
print("Writing clusters aisle rules to csv...")
association_rule_no_clustering.to_csv('../input/associationRule_no_cluster/all_aisle_rules.csv')

Writing clusters aisle rules to csv...


In [13]:
# separate order data by cluster and transform into expected format of group association function at aisle level

cluster_aisle_orders = {}
for i in range(0,9):
    x = orders[orders['cluster'] == i]
    x = x.groupby(['order_id','aisle_id'])['product_id'].count().reset_index().set_index('order_id')['aisle_id'].rename('item_id')
    cluster_aisle_orders[i] = x

In [14]:
# format item association rule dataframes and merge product names
REBUILD_TEST_DATA = False
test_data_path = "../input/associationRule/group0_aisle_rules.csv"
if REBUILD_TEST_DATA or not Path(test_data_path).exists():
    group_aisle_association_rules_dic = {}
    for x in range(0,9):
        group_aisle_association_rules_dic[x] = apriori.association_rules(cluster_aisle_orders[x],.0001)
    for x in group_aisle_association_rules_dic:
        group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x].merge(aisles[['aisle_id','aisle']]
                   .rename(columns = {'aisle_id':'item_A','aisle':'aisle_name_A'}), on = 'item_A')
        group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x].merge(aisles[['aisle_id','aisle']]
                   .rename(columns = {'aisle_id':'item_B','aisle':'aisle_name_B'}), on = 'item_B')
        group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x][['item_A','item_B','aisle_name_A',
                                                                        'aisle_name_B','freqAB','supportAB','freqA',
                                                                        'supportA','freqB','supportB','confidenceAtoB',
                                                                        'confidenceBtoA','lift']]
    group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x].rename(columns = {'item_A':'aisle_A','item_B':'aisle_B'})
    for x in group_aisle_association_rules_dic:
        print("Writing cluster " + str(x) + "'s aisle rules to csv...")
        group_aisle_association_rules_dic[x].to_csv('../input/associationRule/group'+str(x)+'_aisle_rules.csv')

Starting order_item:                1068058
Items with support >= 0.0001:             134
Remaining order_item:               1068058
Remaining orders with 2+ items:      154657
Remaining order_item:               1058033
Item pairs:                            8756
Item pairs with support >= 0.0001:       6961

Starting order_item:                7978050
Items with support >= 0.0001:             134
Remaining order_item:               7978050
Remaining orders with 2+ items:      892495
Remaining order_item:               7951516
Item pairs:                            8905
Item pairs with support >= 0.0001:       7664

Starting order_item:                 401306
Items with support >= 0.0001:             118
Remaining order_item:                401265
Remaining orders with 2+ items:       94134
Remaining order_item:                382173
Item pairs:                            5692
Item pairs with support >= 0.0001:       3318

Starting order_item:                6889543
Items with suppor